In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [72]:
import numpy as np
from scipy.spatial import distance

from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py
import pandas as pd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

def load_file(path, name):
    import importlib.util
    spec = importlib.util.spec_from_file_location(name, path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

models = load_file("../../utils/models.py", "models")
data = load_file("../../utils/data.py", "data")


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../../data/metadata.json'))
tqdm.pandas()

In [3]:
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_ref = pickle.load(open('../../data/infos_pca128.pytorch', 'rb'))
movie_ref_tensor, key_to_id, id_to_key = data.make_movie_tensor(movie_ref)
# download ml20m dataset yourself
ratings = pd.read_csv('../../data/ml-20m/ratings.csv')
user_dict, users = data.prepare_tensor(ratings, key_to_id, frame_size)
del ratings
gc.collect()
clear_output(True)
clear_output(True)
print('Done!')

Done!


In [4]:
ddpg = models.Actor(1290, 128, 256).to(cuda)
td3 = models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../../models/DDPG/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../../models/TD3/td3_policy_1.pt'))

<All keys matched successfully>

In [10]:
test_users = users[-5000:]
batch_size = 25

def prepare_batch_wrapper(x):
    batch = data.prepare_batch(x, movie_ref_tensor, frame_size=frame_size)
    return batch

test_user_dataset = data.UserDataset(test_users, user_dict)
test_dataloader = DataLoader(test_user_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=4,collate_fn=prepare_batch_wrapper)
test_batch = next(iter(test_dataloader))
state = test_batch[0].to(cuda)

In [30]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}
del movie_ref[0]

KeyError: 0

In [34]:
def rank(gen_action, metric):
    scores = []
    for i in movie_ref.keys():
        scores.append([i, metric(movie_ref[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [35]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [36]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,44731,13.751751,Stay Alive,Horror Thriller,en,2006,5.4
1,103731,13.809052,An Angel Named Billy,Drama,en,2007,2.5
2,7474,13.811282,The Boys in the Band,Drama,en,1970,7.0
3,961,13.857841,Little Lord Fauntleroy,Drama Family,en,1936,6.4
4,427,13.992612,Boxing Helena,Drama Romance,en,1993,4.7
5,26188,14.002238,The Heart Is a Lonely Hunter,Drama,en,1968,7.8
6,5766,14.002769,Madman,Horror Thriller,en,1982,5.2
7,95427,14.007966,The Silent Scream,Horror Thriller,en,1979,5.9
8,79547,14.065639,2:13,Horror Thriller Mystery,en,2009,5.0
9,7127,14.088521,Run,Action Thriller Drama,en,1991,5.7


In [37]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,121259,0.669091,The Tattooist,Thriller Horror,en,2007,5.2
1,86815,0.673583,Soul Surfer,Family Action Drama,en,2011,7.0
2,93522,0.676345,Captain Midnight,Adventure Action,en,1942,0.0
3,59715,0.679368,Shutter,Horror Mystery Thriller,en,2008,5.4
4,31692,0.679412,Uncle Nino,Comedy Drama Family,en,2003,6.0
5,115080,0.680945,The Odd Couple,Action Comedy,cn,1979,6.8
6,421,0.685226,Black Beauty,Action Adventure Drama Family,en,1994,6.3
7,4837,0.686175,Cruising,Horror Action Drama Mystery Thriller,en,1980,6.3
8,112938,0.686561,The Detective,Drama Action Thriller Mystery,cn,2007,5.8
9,1685,0.695277,"I Love You, I Love You Not",Drama Family Romance,en,1996,4.8


In [38]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,121259,0.668200,The Tattooist,Thriller Horror,en,2007,5.2
1,31692,0.673043,Uncle Nino,Comedy Drama Family,en,2003,6.0
2,86815,0.673302,Soul Surfer,Family Action Drama,en,2011,7.0
3,59715,0.673730,Shutter,Horror Mystery Thriller,en,2008,5.4
4,93522,0.676451,Captain Midnight,Adventure Action,en,1942,0.0
5,115080,0.680319,The Odd Couple,Action Comedy,cn,1979,6.8
6,421,0.683674,Black Beauty,Action Adventure Drama Family,en,1994,6.3
7,112938,0.686174,The Detective,Drama Action Thriller Mystery,cn,2007,5.8
8,4837,0.686304,Cruising,Horror Action Drama Mystery Thriller,en,1980,6.3
9,1685,0.694257,"I Love You, I Love You Not",Drama Family Romance,en,1996,4.8


In [39]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,130083,76.582862,Kidnapping Mr. Heineken,Drama Action Crime Thriller Mystery,en,2015,5.9
1,6987,78.274383,The Cabinet of Dr. Caligari,Drama Horror Thriller Crime,de,1920,7.8
2,72369,78.773735,Goemon,Action Drama Fantasy,ja,2009,6.7
3,30856,79.425770,Rebels of the Neon God,Drama,zh,1993,7.1
4,1907,79.788915,Mulan,Animation Family Adventure,en,1998,7.9
5,3455,79.823280,Buddy Boy,Drama Thriller Romance,en,1999,5.0
6,59715,79.904216,Shutter,Horror Mystery Thriller,en,2008,5.4
7,96612,80.266611,Compliance,Drama Thriller Crime Mystery,en,2012,6.4
8,120482,80.622039,Vincent: A Life in Color,Documentary,en,2008,0.0
9,6563,80.767237,Masked and Anonymous,Drama Musical,en,2003,5.5


In [40]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,44731,13.751751,Stay Alive,Horror Thriller,en,2006,5.4
1,103731,13.809052,An Angel Named Billy,Drama,en,2007,2.5
2,7474,13.811282,The Boys in the Band,Drama,en,1970,7.0
3,961,13.857841,Little Lord Fauntleroy,Drama Family,en,1936,6.4
4,427,13.992612,Boxing Helena,Drama Romance,en,1993,4.7
5,26188,14.002238,The Heart Is a Lonely Hunter,Drama,en,1968,7.8
6,5766,14.002769,Madman,Horror Thriller,en,1982,5.2
7,95427,14.007966,The Silent Scream,Horror Thriller,en,1979,5.9
8,79547,14.065639,2:13,Horror Thriller Mystery,en,2009,5.0
9,7127,14.088521,Run,Action Thriller Drama,en,1991,5.7


In [41]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,3333,2.745283,The Killing of Sister George,Comedy Drama,en,1968,6.8
1,107215,2.822541,Midnight Clear,Drama,en,2006,6.5
2,6052,2.846914,The Horse in the Gray Flannel Suit,Drama Family Comedy,en,1968,5.3
3,3853,2.858798,The Tic Code,,en,2000,4.5
4,33621,2.864920,Somersault,Drama Romance,en,2004,6.3
5,102479,2.866989,Reincarnated,Musical Documentary,en,2012,6.3
6,463,2.868352,Guilty as Sin,Drama Thriller Mystery,en,1993,5.7
7,1433,2.872989,The Machine,Horror Thriller Sci-Fi,en,1994,5.3
8,85527,2.875412,Putty Hill,Drama,en,2010,5.9
9,4505,2.877447,For Keeps,Drama Family Romance,en,1988,6.1


In [42]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,59715,0.687142,Shutter,Horror Mystery Thriller,en,2008,5.4
1,103647,0.713143,The Surgeon,Horror Sci-Fi,fr,1995,6.1
2,49530,0.721009,Blood Diamond,Drama Thriller Action,en,2006,7.5
3,1685,0.722459,"I Love You, I Love You Not",Drama Family Romance,en,1996,4.8
4,86815,0.723578,Soul Surfer,Family Action Drama,en,2011,7.0
5,115080,0.728468,The Odd Couple,Action Comedy,cn,1979,6.8
6,4837,0.730294,Cruising,Horror Action Drama Mystery Thriller,en,1980,6.3
7,54270,0.731793,Skinwalkers,Fantasy Action Horror,en,2006,4.5
8,49435,0.733367,Reeker,Horror Mystery,en,2005,5.3
9,86758,0.733907,The Bastards,Drama Thriller,en,2008,6.5


In [43]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,103731,124.678558,An Angel Named Billy,Drama,en,2007,2.5
1,961,125.308823,Little Lord Fauntleroy,Drama Family,en,1936,6.4
2,59715,125.578674,Shutter,Horror Mystery Thriller,en,2008,5.4
3,79547,125.606918,2:13,Horror Thriller Mystery,en,2009,5.0
4,115080,125.659149,The Odd Couple,Action Comedy,cn,1979,6.8
5,101581,125.698349,Angel Dog,Drama Family,en,2011,3.6
6,30856,125.750404,Rebels of the Neon God,Drama,zh,1993,7.1
7,86758,125.886299,The Bastards,Drama Thriller,en,2008,6.5
8,80906,126.372276,Inside Job,Documentary,en,2010,7.8
9,5766,126.421944,Madman,Horror Thriller,en,1982,5.2


# TD3

In [44]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [45]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,7933,10.426373,A Night in Casablanca,Comedy,en,1946,7.2
1,5977,10.454474,Where’s Poppa?,Comedy,en,1970,6.8
2,69487,10.469413,Anna Lucasta,Drama,en,1958,5.7
3,73526,10.507264,A Matter of Dignity,Drama,el,1958,7.0
4,8893,10.511228,Basket Case 2,Horror,en,1990,5.6
5,63862,10.516020,Revenge of the Zombies,Horror,en,1943,5.0
6,33917,10.555430,The Member of the Wedding,Drama,en,1952,6.9
7,98805,10.568145,Miss Annie Rooney,Drama,en,1942,5.8
8,31619,10.597951,Going All the Way,Drama,en,1997,4.7
9,25912,10.656021,The Enchanted Cottage,Drama Romance,en,1945,7.1


In [46]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,89977,0.614526,Don't Change Your Husband,Comedy,en,1919,7.0
1,3630,0.657698,Lisa and the Devil,Fantasy Horror Sci-Fi,it,1973,6.3
2,98805,0.671160,Miss Annie Rooney,Drama,en,1942,5.8
3,4027,0.675339,"O Brother, Where Art Thou?",Action Adventure Comedy,en,2000,7.3
4,93963,0.682378,Wild About Harry,Comedy Romance Drama,en,2000,4.3
5,106749,0.683700,Mr. Morgan's Last Love,Drama Comedy,en,2013,6.8
6,40033,0.688078,The Adventures of Prince Achmed,Drama Animation Adventure Fantasy,de,1926,7.8
7,429,0.688602,Cabin Boy,Comedy Fantasy Adventure,en,1994,5.0
8,38701,0.691781,Don't Come Knocking,Drama,en,2005,6.6
9,113020,0.693582,Two Tars,Comedy,en,1928,6.7


In [47]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,89977,0.618633,Don't Change Your Husband,Comedy,en,1919,7.0
1,3630,0.648607,Lisa and the Devil,Fantasy Horror Sci-Fi,it,1973,6.3
2,4027,0.672030,"O Brother, Where Art Thou?",Action Adventure Comedy,en,2000,7.3
3,40033,0.677453,The Adventures of Prince Achmed,Drama Animation Adventure Fantasy,de,1926,7.8
4,98805,0.677827,Miss Annie Rooney,Drama,en,1942,5.8
5,93963,0.678019,Wild About Harry,Comedy Romance Drama,en,2000,4.3
6,106749,0.684878,Mr. Morgan's Last Love,Drama Comedy,en,2013,6.8
7,38701,0.687736,Don't Come Knocking,Drama,en,2005,6.6
8,58047,0.690582,"Definitely, Maybe",Comedy Romance,en,2008,6.8
9,113020,0.690616,Two Tars,Comedy,en,1928,6.7


In [48]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,38701,77.869147,Don't Come Knocking,Drama,en,2005,6.6
1,26022,77.982207,Man of a Thousand Faces,Drama,en,1957,6.1
2,128846,78.018729,Robin Williams: An Evening with Robin Williams,Comedy,en,1983,7.3
3,92522,78.066729,Frisco Jenny,Drama,en,1932,7.0
4,59339,78.186790,Mister Lonely,Comedy Drama,en,2008,6.8
5,123419,78.434217,Private Detective 62,Crime Drama,en,1933,6.7
6,92231,78.436194,The Blue Bird,Drama Fantasy,en,1918,5.7
7,828,78.527550,The Adventures of Pinocchio,Animation Family Fantasy,en,1996,5.0
8,114042,78.676145,Third Person,Drama Romance,en,2013,5.9
9,105073,78.829608,Officer Down,Crime Drama,en,2013,5.8


In [49]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,7933,10.426373,A Night in Casablanca,Comedy,en,1946,7.2
1,5977,10.454474,Where’s Poppa?,Comedy,en,1970,6.8
2,69487,10.469413,Anna Lucasta,Drama,en,1958,5.7
3,73526,10.507264,A Matter of Dignity,Drama,el,1958,7.0
4,8893,10.511228,Basket Case 2,Horror,en,1990,5.6
5,63862,10.516020,Revenge of the Zombies,Horror,en,1943,5.0
6,33917,10.555430,The Member of the Wedding,Drama,en,1952,6.9
7,98805,10.568145,Miss Annie Rooney,Drama,en,1942,5.8
8,31619,10.597951,Going All the Way,Drama,en,1997,4.7
9,25912,10.656021,The Enchanted Cottage,Drama Romance,en,1945,7.1


In [50]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,6830,2.135513,Sudden Fear,Drama Thriller,en,1952,7.3
1,26141,2.166608,The Loved One,Comedy,en,1965,6.4
2,8893,2.188493,Basket Case 2,Horror,en,1990,5.6
3,8758,2.191659,Kotch,Comedy Drama,en,1971,6.8
4,26226,2.191885,"Hi, Mom!",Comedy Drama,en,1970,5.8
5,26080,2.216591,Eegah,Horror,en,1962,2.8
6,7933,2.223791,A Night in Casablanca,Comedy,en,1946,7.2
7,26356,2.229031,Supervixens,Comedy,en,1975,6.4
8,26402,2.231717,Harper Valley P.T.A.,Comedy,en,1978,7.0
9,117606,2.257692,Divorce,Drama,en,1945,4.0


In [51]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,59339,0.675832,Mister Lonely,Comedy Drama,en,2008,6.8
1,89977,0.678051,Don't Change Your Husband,Comedy,en,1919,7.0
2,26022,0.698923,Man of a Thousand Faces,Drama,en,1957,6.1
3,3630,0.700159,Lisa and the Devil,Fantasy Horror Sci-Fi,it,1973,6.3
4,105073,0.708607,Officer Down,Crime Drama,en,2013,5.8
5,38701,0.709373,Don't Come Knocking,Drama,en,2005,6.6
6,828,0.709863,The Adventures of Pinocchio,Animation Family Fantasy,en,1996,5.0
7,4512,0.711038,Hot to Trot,Comedy,en,1988,5.1
8,9012,0.712276,Ruby Gentry,Drama Romance,en,1952,6.6
9,40033,0.713282,The Adventures of Prince Achmed,Drama Animation Adventure Fantasy,de,1926,7.8


In [52]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,73526,92.109741,A Matter of Dignity,Drama,el,1958,7.0
1,33917,93.695114,The Member of the Wedding,Drama,en,1952,6.9
2,124779,93.907562,Home of the Brave,Drama,en,1949,6.8
3,3127,94.069412,Holy Smoke,Comedy Drama,en,1999,5.5
4,31619,94.178787,Going All the Way,Drama,en,1997,4.7
5,31047,94.244980,Jinxed!,Comedy,en,1982,4.3
6,70889,94.383339,The Pom Pom Girls,Comedy,en,1976,5.4
7,83357,94.768585,A Gathering of Eagles,Drama Romance,en,1963,6.4
8,63862,94.866096,Revenge of the Zombies,Horror,en,1943,5.0
9,59138,95.475861,Bernard and Doris,Comedy Drama,en,2006,6.2


# BCQ

In [59]:
perturbator = models.bcqPerturbator(1290, 128, 256).to(cuda)
generator = models.bcqGenerator(1290, 128, 512).to(cuda)

perturbator.load_state_dict(torch.load('../../models/BCQ/bcq_policy.pt'))
generator.load_state_dict(torch.load('../../models/BCQ/bcq_vae.pt'))

<All keys matched successfully>

In [60]:
sampled_actions = generator.decode(state)
perturbed_actions= perturbator(state, sampled_actions)

In [63]:
bcq_action = perturbed_actions[np.random.randint(0,
                               sampled_actions.size(0), 1)[0]].detach().cpu().numpy()

In [64]:
rank(bcq_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,65738,10.148527,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
1,122290,10.228902,Homeboy,Drama,en,1988,4.5
2,72153,10.258544,A Dangerous Woman,Drama Romance,en,1993,5.2
3,1170,10.386846,Best of the Best 3: No Turning Back,Action,en,1995,5.7
4,961,10.409158,Little Lord Fauntleroy,Drama Family,en,1936,6.4
5,26141,10.416187,The Loved One,Comedy,en,1965,6.4
6,106540,10.421478,Delivery Man,Comedy,en,2013,6.2
7,32074,10.450201,Kiss The Bride,Comedy Drama Romance,en,2002,5.4
8,90576,10.456925,What's Your Number?,Comedy Romance,en,2011,6.4
9,72,10.487558,Kicking and Screaming,Comedy Drama Romance,en,1995,6.9


In [65]:
rank(bcq_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,52973,0.700067,Knocked Up,Comedy Romance Drama,en,2007,6.2
1,3968,0.701145,Bedazzled,Fantasy Comedy Romance,en,2000,5.8
2,108709,0.715932,Generation Kill,Drama War,en,2008,8.6
3,4995,0.719721,A Beautiful Mind,Drama Romance,en,2001,7.8
4,158,0.725523,Casper,Fantasy Comedy Family,en,1995,6.2
5,63131,0.727749,Role Models,Comedy,en,2008,6.4
6,63793,0.730138,Wagon Master,Action Western,en,1950,6.3
7,79091,0.734288,Despicable Me,Animation Family Comedy,en,2010,7.2
8,1126,0.738763,Drop Dead Fred,Comedy Fantasy Family,en,1991,6.1
9,100505,0.739323,Monte Walsh,Action Western,en,2003,5.8


In [66]:
rank(bcq_action, distance.correlation)

,id,score,title,genres,language,year,rating
0,52973,0.700144,Knocked Up,Comedy Romance Drama,en,2007,6.2
1,3968,0.701208,Bedazzled,Fantasy Comedy Romance,en,2000,5.8
2,108709,0.710985,Generation Kill,Drama War,en,2008,8.6
3,4995,0.718374,A Beautiful Mind,Drama Romance,en,2001,7.8
4,158,0.725611,Casper,Fantasy Comedy Family,en,1995,6.2
5,63131,0.727765,Role Models,Comedy,en,2008,6.4
6,63793,0.730111,Wagon Master,Action Western,en,1950,6.3
7,79091,0.734324,Despicable Me,Animation Family Comedy,en,2010,7.2
8,100505,0.737390,Monte Walsh,Action Western,en,2003,5.8
9,1126,0.737648,Drop Dead Fred,Comedy Fantasy Family,en,1991,6.1


In [67]:
rank(bcq_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,114849,75.923258,The Prince,Thriller Action,en,2014,4.8
1,98085,76.051232,Labor Pains,Comedy,en,2009,5.3
2,122290,76.569721,Homeboy,Drama,en,1988,4.5
3,85106,77.413073,The Assassin Next Door,Action Drama Thriller,ru,2009,5.5
4,961,77.475821,Little Lord Fauntleroy,Drama Family,en,1936,6.4
5,100583,77.973128,Arnulf Rainer,,xx,1960,4.6
6,6470,78.060276,Chisum,Western,en,1970,6.7
7,103576,78.102289,Chu Chin Chow,Musical Fantasy Romance,en,1934,0.0
8,2558,78.260270,Forces of Nature,Romance Comedy,en,1999,5.4
9,103435,78.892301,"Ah, Wilderness!",Comedy Drama,en,1935,5.5


In [68]:
rank(bcq_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,65738,10.148527,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
1,122290,10.228902,Homeboy,Drama,en,1988,4.5
2,72153,10.258544,A Dangerous Woman,Drama Romance,en,1993,5.2
3,1170,10.386846,Best of the Best 3: No Turning Back,Action,en,1995,5.7
4,961,10.409158,Little Lord Fauntleroy,Drama Family,en,1936,6.4
5,26141,10.416187,The Loved One,Comedy,en,1965,6.4
6,106540,10.421478,Delivery Man,Comedy,en,2013,6.2
7,32074,10.450201,Kiss The Bride,Comedy Drama Romance,en,2002,5.4
8,90576,10.456925,What's Your Number?,Comedy Romance,en,2011,6.4
9,72,10.487558,Kicking and Screaming,Comedy Drama Romance,en,1995,6.9


In [69]:
rank(bcq_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,105042,2.072893,The Wedding Party,Comedy Romance,en,1969,4.9
1,542,2.096572,Son in Law,Comedy Drama Romance,en,1993,5.8
2,114457,2.121568,Cold Turkey,Comedy,en,1971,7.2
3,67087,2.164096,"I Love You, Man",Comedy,en,2009,6.5
4,4481,2.164193,The Boost,Drama Romance,en,1988,5.7
5,78,2.164253,The Crossing Guard,Drama Thriller,en,1995,6.0
6,2506,2.169194,The Other Sister,Comedy Drama Romance,en,1999,6.3
7,4168,2.176382,Get Over It,Comedy Romance,en,2001,5.4
8,55389,2.176580,The Nanny,Drama Thriller,en,1965,7.1
9,2369,2.177724,Desperately Seeking Susan,Comedy,en,1985,6.1


In [70]:
rank(bcq_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,122290,0.700850,Homeboy,Drama,en,1988,4.5
1,85106,0.729621,The Assassin Next Door,Action Drama Thriller,ru,2009,5.5
2,1126,0.731436,Drop Dead Fred,Comedy Fantasy Family,en,1991,6.1
3,961,0.732149,Little Lord Fauntleroy,Drama Family,en,1936,6.4
4,3968,0.737227,Bedazzled,Fantasy Comedy Romance,en,2000,5.8
5,7563,0.739252,The Discovery of Heaven,Drama Fantasy,en,2001,7.6
6,114849,0.740137,The Prince,Thriller Action,en,2014,4.8
7,55190,0.741576,Love and Other Disasters,Comedy Romance,en,2006,5.9
8,65762,0.743002,Soup to Nuts,Comedy Romance,en,1930,6.3
9,63131,0.743369,Role Models,Comedy,en,2008,6.4


In [71]:
rank(bcq_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,122290,86.565430,Homeboy,Drama,en,1988,4.5
1,65738,90.176620,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
2,72153,90.435600,A Dangerous Woman,Drama Romance,en,1993,5.2
3,961,90.634674,Little Lord Fauntleroy,Drama Family,en,1936,6.4
4,26141,91.761215,The Loved One,Comedy,en,1965,6.4
5,130604,92.452560,Maid in Sweden,Drama,en,1971,4.2
6,32584,92.805542,The Ballad of Jack and Rose,Drama,en,2005,6.4
7,2500,93.328110,Jawbreaker,Comedy,en,1999,5.9
8,26356,93.373871,Supervixens,Comedy,en,1975,6.4
9,94204,93.397293,Androcles and the Lion,Comedy,en,1952,6.3
